In [61]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [83]:
# @hidden cell
CLIENT_ID = 'KRBOQIRZJFXTS2BGSK2CN0YYB2CGYGFA1OUN1HMDEKSZUMPP' # your Foursquare ID
CLIENT_SECRET = 'C1V3DPD45QVRMHUKMUGBYHSRTJE4S525EVT0AUAXC3GB40YO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')


Your credentails:


In [84]:
address = '1000 Brussels'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

50.8465573 4.351697


In [85]:
search_query = 'italian'
radius = 1000
print(search_query + ' .... OK!')

italian .... OK!


In [86]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=KRBOQIRZJFXTS2BGSK2CN0YYB2CGYGFA1OUN1HMDEKSZUMPP&client_secret=C1V3DPD45QVRMHUKMUGBYHSRTJE4S525EVT0AUAXC3GB40YO&ll=50.8465573,4.351697&v=20180604&query=italian&radius=1000&limit=30'

In [87]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ce9470d351e3d128901f29d'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_boutique_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d104951735',
      'name': 'Boutique',
      'pluralName': 'Boutiques',
      'primary': True,
      'shortName': 'Boutique'}],
    'hasPerk': False,
    'id': '54dcae32498e197039753826',
    'location': {'address': 'Rue des Teinturiers 15',
     'cc': 'BE',
     'city': 'Brussel',
     'country': 'België',
     'distance': 155,
     'formattedAddress': ['Rue des Teinturiers 15', '1000 Brussel', 'België'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 50.84568038587185,
       'lng': 4.3499765390879475}],
     'lat': 50.84568038587185,
     'lng': 4.3499765390879475,
     'postalCode': '1000',
     'state': 'Brussels Hoofdstedelijk Gewest'},
    'name': 'KIKIAMO Italian Style',
    'referralId': 'v-1558791949'},
   {'categories': [{

In [88]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'primary': True, 'shortName': 'Boutique', 'n...",False,54dcae32498e197039753826,Rue des Teinturiers 15,BE,Brussel,België,155,"[Rue des Teinturiers 15, 1000 Brussel, België]","[{'lat': 50.84568038587185, 'lng': 4.349976539...",50.845680,4.349977,NaN,1000,Brussels Hoofdstedelijk Gewest,KIKIAMO Italian Style,v-1558791949
1,"[{'primary': True, 'shortName': 'Diner', 'name...",False,5198e914498eccd5bca6b268,NaN,BE,NaN,België,287,[België],"[{'lat': 50.847684, 'lng': 4.348022, 'label': ...",50.847684,4.348022,NaN,NaN,NaN,Italiano Vero,v-1558791949
2,"[{'primary': True, 'shortName': 'Italian', 'na...",False,4fce634ae4b083992e3a61ae,28 Greepstraat,BE,Brussel,België,224,"[28 Greepstraat, Brussel, België]","[{'lat': 50.84835918734941, 'lng': 4.353120921...",50.848359,4.353121,NaN,NaN,Brussels Hoofdstedelijk Gewest,L'italiano,v-1558791949
3,"[{'primary': True, 'shortName': 'Sandwiches', ...",False,4b45e01bf964a520821126e3,Nijverheidsstraat 26 Rue de l'Industrie,BE,Brussel,België,1363,"[Nijverheidsstraat 26 Rue de l'Industrie, 1000...","[{'lat': 50.84274637782242, 'lng': 4.370136272...",50.842746,4.370136,European Quarter,1000,Brussels Hoofdstedelijk Gewest,Italiano n°1 Paninoteca,v-1558791949


In [89]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,KIKIAMO Italian Style,Boutique,Rue des Teinturiers 15,BE,Brussel,België,155,"[Rue des Teinturiers 15, 1000 Brussel, België]","[{'lat': 50.84568038587185, 'lng': 4.349976539...",50.845680,4.349977,NaN,1000,Brussels Hoofdstedelijk Gewest,54dcae32498e197039753826
1,Italiano Vero,Diner,NaN,BE,NaN,België,287,[België],"[{'lat': 50.847684, 'lng': 4.348022, 'label': ...",50.847684,4.348022,NaN,NaN,NaN,5198e914498eccd5bca6b268
2,L'italiano,Italian Restaurant,28 Greepstraat,BE,Brussel,België,224,"[28 Greepstraat, Brussel, België]","[{'lat': 50.84835918734941, 'lng': 4.353120921...",50.848359,4.353121,NaN,NaN,Brussels Hoofdstedelijk Gewest,4fce634ae4b083992e3a61ae
3,Italiano n°1 Paninoteca,Sandwich Place,Nijverheidsstraat 26 Rue de l'Industrie,BE,Brussel,België,1363,"[Nijverheidsstraat 26 Rue de l'Industrie, 1000...","[{'lat': 50.84274637782242, 'lng': 4.370136272...",50.842746,4.370136,European Quarter,1000,Brussels Hoofdstedelijk Gewest,4b45e01bf964a520821126e3


In [90]:
dataframe_filtered.name

0      KIKIAMO Italian Style
1              Italiano Vero
2                 L'italiano
3    Italiano n°1 Paninoteca
Name: name, dtype: object

In [91]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Grand Place',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [92]:
venue_id = '5198e914498eccd5bca6b268' # ID of  Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/5198e914498eccd5bca6b268?client_id=KRBOQIRZJFXTS2BGSK2CN0YYB2CGYGFA1OUN1HMDEKSZUMPP&client_secret=C1V3DPD45QVRMHUKMUGBYHSRTJE4S525EVT0AUAXC3GB40YO&v=20180604'

In [93]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['likes', 'hereNow', 'canonicalUrl', 'name', 'beenHere', 'categories', 'tips', 'timeZone', 'allowMenuUrlEdit', 'stats', 'createdAt', 'contact', 'specials', 'price', 'location', 'ok', 'dislike', 'listed', 'attributes', 'inbox', 'reasons', 'shortUrl', 'id', 'pageUpdates', 'photos', 'verified'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '€', 'priceTier': 1}],
    'name': 'Price',
    'summary': '€',
    'type': 'price'}]},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'canonicalUrl': 'https://foursquare.com/v/italiano-vero/5198e914498eccd5bca6b268',
 'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_',
    'suffix': '.png'},
   'id': '4bf58dd8d48988d147941735',
   'name': 'Diner',
   'pluralName': 'Diners',
   'primary': True,
   'shortName': 'Diner'}],
 'contact': {},
 'createdAt': 1368975636,
 'dislike': False,
 'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
 'id': '5198e914498eccd5bca6b268',
 'inbox': {'count': 0, 'items': []},
 'likes': {'count': 0, 'groups': []},
 'listed': {'count': 0,
  'groups': [{'count': 0,
    'items': [],
    'name': 'Lists from other people',
    'type': 'others

In [94]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [95]:
venue_id = '4fce634ae4b083992e3a61ae' # ID of  Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fce634ae4b083992e3a61ae?client_id=KRBOQIRZJFXTS2BGSK2CN0YYB2CGYGFA1OUN1HMDEKSZUMPP&client_secret=C1V3DPD45QVRMHUKMUGBYHSRTJE4S525EVT0AUAXC3GB40YO&v=20180604'

In [96]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['likes', 'hereNow', 'canonicalUrl', 'name', 'beenHere', 'categories', 'bestPhoto', 'tips', 'timeZone', 'allowMenuUrlEdit', 'colors', 'stats', 'createdAt', 'contact', 'specials', 'price', 'location', 'ok', 'dislike', 'listed', 'attributes', 'inbox', 'reasons', 'shortUrl', 'id', 'pageUpdates', 'photos', 'verified'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '€€', 'priceTier': 2}],
    'name': 'Price',
    'summary': '€€',
    'type': 'price'},
   {'count': 5,
    'items': [{'displayName': 'Credit Cards', 'displayValue': 'Yes'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'}]},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'bestPhoto': {'createdAt': 1463676979,
  'height': 540,
  'id': '573df033498e3b7674000bce',
  'prefix': 'https://fastly.4sqi.net/img/general/',
  'source': {'name': 'Swarm for Android', 'url': 'https://www.swarmapp.com'},
  'suffix': '/121334459_JeqotJgOvyqm9roXqzzboURNalIQo7nHQHMB8lel2F8.jpg',
  'visibility': 'public',
  'width': 960},
 'canonicalUrl': 'https://foursquare.com/v/litaliano/4fce634ae4b083992e3a61ae',
 'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
    '

In [97]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.
